In [11]:
def crear_datos():
  #Descargar archivo  
  #Open, readlines, close
  r = open("oneline.txt")
  ls = r.readlines()
  r.close()

  #Leer numero de numeros y medidas
  n_nums = int(ls.pop(0).strip())
  n_fils = int(ls.pop(0).strip())
  n_cols = int(ls.pop(0).strip())

  DAT = []
  RES = []
  #Procesar numero a numero
  for n in range(0,n_nums):
    #Numero correcto en string
    sres = ls.pop(0).strip()
    #Dibujo del numero en string
    sdat = ls.pop(0).strip()

    res = convertir_res(sres)
    RES.append(res)
    dat = convertir_dat(sdat)
    DAT.append(dat)

  return DAT,RES

def convertir_res(sres):
  res = []
  for z in range(0,N_Z):
    res.append(0.0)
  res[int(sres)] = 1.0
  return res

def convertir_dat(sdat):
  dat = []
  for c in sdat:
    dat.append(int(c))
  return dat


In [12]:
def crear_red():
  x = []          # primera capa
  for i in range(0,N_X):
    x.append(0)

  w1 = []       #segunda capa
  for y in range(0,N_Y):
    W = []
    for x in range(0,N_X):
      r = (random.randint(0,100)/100)-0.5
      W.append(r)
    w1.append(W)


  #b1 #primer bias
  b1 = []
  for y in range(0, N_Y):
    b1.append((random.randint(0,100)/100)-0.5)

  #b2      #segundo bias 
  b2 = []
  for z in range(0, N_Z):
    b2.append((random.randint(0,100)/100)-0.5)




  w2 = []       # pesos tercera capa
  for z in range(0,N_Z):
    W = []
    for x in range(0,N_Y):
      r = (random.randint(0,100)/100)-0.5
      W.append(r)
    w2.append(W)



  return x, w1, w2,b1, b2

# New Section

In [13]:
def get_predictions(A2):
  maxpos = -1
  maxval = -1.0
  for z in range(0,N_Z):
    if A2[z] > maxval:
      maxval = A2[z]
      maxpos = z
  return maxpos


In [25]:
def entrenar(nums,x,  w1, w2,b1, b2, start):
  
  #Para cada numero de mis datos
  END = 20000
  for i in range(nums):
    PRECITION = 0
    NUMBER = 0
    for n in range(0,END):
      x = DAT[n]
      y = RES[n]

      Z1, A1, Z2, A2  = forward(x, w1, b1, w2, b2)      # generamos el output
      dW1, db1, dW2, db2 = back(Z1, A1, A2,  w2, x, y)  # back propagation
      w1, b1, w2, b2 = actualizar(w1,b1, w2,b2, dW1, dW2,db1, db2, LRATE,END)   #update weights
      if i%10 == 0:
        if A2.index(max(A2)) == y.index(max(y)):
          PRECITION +=1
        NUMBER +=1
    if i %10 == 0:
      stop = timeit.default_timer()
      print("tiempo entre rondas:","{:.2f}".format(stop-start),'segundos' )
      start = stop
      print("Round",i)
      print("average hasta ahora de:" ,PRECITION/NUMBER)
      PRECITION = 0
      NUMBER = 0
  return w1, b1, w2, b2

In [15]:
def forward(x, w1, b1, w2, b2):    #FORWARD PROPAGATION
  Z1, A1 = forwardX2Y(x,w1,b1)
  Z2, A2 = forwardY2Z(w2,A1,b2)
  return Z1, A1, Z2, A2

def forwardX2Y(x,w1,b1):
  Z1 = []
  A1 = []
  for y in range(0,N_Y):
    z1, a1 =  forwardX2Yone(x,y,w1,b1)
    Z1.append(z1)
    A1.append(a1)
  return Z1, A1   #A1 es el output de la segunda capa

def forwardX2Yone(x,y,w1,b1):
  activation = 0
  for i in range(0,N_X):
    activation += x[i]*w1[y][i]

  activation += b1[y]
  return activation, max(activation, 0)


def forwardY2Z(w2,A1,b2):
  Z2 = []
  A2 = []
  for z in range(0,N_Z):
    z2 = forwardY2Zone(z,w2,A1,b2)
    Z2.append(z2)
  A2 = softmax(Z2, A2)
  return Z2, A2 #A2 es el output de la tercera capa
  
def forwardY2Zone(z,w2,A1, b2):
  activation = 0
  
  for i in range(0,N_Y):
    activation += A1[i]*w2[z][i]
  activation += b2[z]   

  return activation


def softmax(Z2,A2):
  A2 = []
  general = 0
  for z in range(0, N_Z):
    general += math.exp(Z2[z])
  for z in range(0, N_Z):
    A2.append(math.exp(Z2[z])/general)
  return A2

In [16]:
def mult_matrix(a,b):
    matrix = []
    for i in range(len(a)):
      matrix.append([])
      for j in range(len(b[0])):
          matrix[i].append(0)

    for i in range(len(a)):
      for j in range(len(b[0])):
        for k in range(len(a[0])):
          matrix[i][j]  += a[i][k] * b[k][j]
    return matrix

def transpose(matrix):

    result = []
    for i in range(len(matrix[0])):
      partial = []
      for j in range(len(matrix)):
        partial.append(0)
      result.append(partial)

    for i in range(len(matrix[0])):
        for j in range(len(matrix)):
            result[i][j] = matrix[j][i] 
    return result

In [17]:

def back(Z1, A1, A2,  w2, x, y):   #back propagation ERROR
  dZ2, dW2 = backZ(A1, A2, y)
  w2T = transpose(w2)
  dW1, dZ1 = backY(x, w2T, Z1, dZ2)
  db2 = 0
  for value in dZ2:
     db2 +=value[0]
  db1 = 0
  for value in dZ1:
     db1 +=value[0]
  return dW1, db1, dW2, db2
  

def backZ( A1,  A2,  y):
  dZ2 = []
  for z in range(0,N_Z):    #ACTUALIZAMOS Z CON LOS OUTPUTS Y SU ERROR
    #Error
    Error = [A2[z] - y[z]]
    dZ2.append(Error)

  definiti = []
  definiti.append(A1)
  dW2 = mult_matrix(dZ2, definiti)
  return dZ2, dW2

def backY(x, w2T, Z1, dZ2):
  dZ1 = mult_matrix(w2T, dZ2)
  for y in range(0,N_Y):  
    if Z1[y]>0:
      dZ1[y] = dZ1[y]
    else:
      dZ1[y] = [0]
  xT = [x]
  dW1 = mult_matrix(dZ1, xT)
  return dW1, dZ1
  



In [26]:
def actualizar(w1,b1, w2,b2, dW1, dW2, db1, db2, LRATE, END):
  w1 = actualizarY(w1, dW1, LRATE, END)
  b1 = actualizarB1(b1,db1, LRATE, END)
  w2 = actualizarZ(w2, dW2, LRATE, END)
  b2 = actualizarB2(b2,db2, LRATE, END)
  return w1, b1,w2,b2

def actualizarB1(b1, db1, LRATE, END):
  for i in range(len(b1)):
    b1[i] = b1[i] - LRATE*db1/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return b1

def actualizarB2(b2, db2, LRATE, END):
  for i in range(len(b2)):
    b2[i] = b2[i] - LRATE*db2/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return b2

def actualizarY(w1, dW1, LRATE, END):
  for y in range(0,N_Y):
    for x in range(0,N_X):
      w1[y][x] -= LRATE *dW1[y][x]/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return w1

def actualizarZ(w2, dW2, LRATE, END):
  for z in range(0,N_Z):
    for y in range(0,N_Y):
      w2[z][y] -= LRATE *dW2[z][y]/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return w2


In [19]:
def predecir(X, w1,b1, w2, b2):
  Z1, A1, Z2, A2 = forward(X,w1, b1, w2,  b2)
  #Buscar que Z es el numero maximo
  maxpos = -1
  maxval = -1.0
  for z in range(0,N_Z):
    if A2[z] > maxval:
      maxval = A2[z]
      maxpos = z
  return maxpos

In [29]:
#CODIGO PRINCIPAL (Part 1)
import random
import math

#Configuracion
N_X = 28*28
N_Y = 40
N_Z = 10 
LRATE = 0.1


#Crear red y datos  !wget -O oneline.txt https://www.dropbox.com/s/ps958u765f2jabe/oneline.txt?dl=0

DAT,RES = crear_datos()

In [30]:
import timeit
#CODIGO PRINCIPAL (Part 2)

start = timeit.default_timer()
#Entrenarx
X,  w1, w2, b1, b2 = crear_red()        
w1, b1,w2,b2 = entrenar(500, X,  w1, w2, b1, b2, start) #probamos con 400 imagenes


#Predecir
X = convertir_dat(
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000001100000000000000000"+
"0000000001100000000000000000"+
"0000000011100000011100000000"+
"0000000011100000011100000000"+
"0000000011100000111000000000"+
"0000000011000000111000000000"+
"0000000111000000111000000000"+
"0000000111000000111000000000"+
"0000000111000001111000000000"+
"0000000111000001111111100000"+
"0000000111111111111111100000"+
"0000000111111111111111000000"+
"0000000001111111110000000000"+
"0000000000000011100000000000"+
"0000000000000011100000000000"+
"0000000000000011100000000000"+
"0000000000000011100000000000"+
"0000000000000111100000000000"+
"0000000000000111100000000000"+
"0000000000000111100000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000")
num = predecir(X,w1,b1,w2,b2)
print ("El numero es un", num, "y era un ", 4)


tiempo entre rondas: 439.97 segundos
Round 0
average hasta ahora de: 0.1336
tiempo entre rondas: 4348.82 segundos
Round 10
average hasta ahora de: 0.43225
tiempo entre rondas: 4273.23 segundos
Round 20
average hasta ahora de: 0.57525
tiempo entre rondas: 4267.38 segundos
Round 30
average hasta ahora de: 0.6481
tiempo entre rondas: 4171.38 segundos
Round 40
average hasta ahora de: 0.6902
tiempo entre rondas: 4986.70 segundos
Round 50
average hasta ahora de: 0.71795
tiempo entre rondas: 5681.03 segundos
Round 60
average hasta ahora de: 0.73815
tiempo entre rondas: 5630.22 segundos
Round 70
average hasta ahora de: 0.75515
tiempo entre rondas: 5689.80 segundos
Round 80
average hasta ahora de: 0.76735
tiempo entre rondas: 5557.46 segundos
Round 90
average hasta ahora de: 0.77895
tiempo entre rondas: 4151.32 segundos
Round 100
average hasta ahora de: 0.78785
tiempo entre rondas: 4181.31 segundos
Round 110
average hasta ahora de: 0.7949
tiempo entre rondas: 5501.70 segundos
Round 120
average 

In [92]:
X_test = DAT[42000:]
y_test = RES[42000:]
number = 0
for i in range(len(y_test)):  
  X = X_test[i]
  E = y_test[i]
  _, _,_, A2 = forward(X,www1, bb1, www2, bb2)
  if A2.index(max(A2)) == E.index(max(E)):
    number +=1
print(number/len(y_test))


0.8711785714285715


In [67]:
w1[1]

[-0.28,
 0.010000000000000009,
 0.48,
 -0.22999999999999998,
 0.15000000000000002,
 -0.2,
 0.49,
 -0.15999999999999998,
 -0.16999999999999998,
 -0.020000000000000018,
 0.19999999999999996,
 -0.030000000000000027,
 -0.38989571100111803,
 -0.299895711001118,
 0.2600776091527738,
 -0.08992239084722613,
 -0.07,
 0.26,
 -0.46,
 0.43000000000000005,
 -0.4,
 0.41000000000000003,
 -0.12,
 0.24,
 0.43999999999999995,
 0.15000000000000002,
 0.050000000000000044,
 0.08999999999999997,
 -0.38,
 0.040000000000000036,
 -0.030000000000000027,
 -0.31,
 0.26,
 -0.4500218956940788,
 0.35952281718418977,
 0.22871932407666942,
 0.2882281910943587,
 -0.3617896614158332,
 -0.4808178849567238,
 0.32934456778423155,
 0.23903840115111516,
 0.3695409807955297,
 0.08945109471331258,
 -0.18966194732195488,
 0.2002879948001984,
 0.3505166386382517,
 -0.36961032295715046,
 -0.5001143085931842,
 0.36971935062184463,
 0.009719322459423021,
 -0.1202470245710893,
 -0.219999441877968,
 -0.27,
 0.09999999999999998,
 0.4,

In [89]:
w1_save = ''
for i in range(len(w1)):
    w1_save += f"{w1[i][0]}" 
    for x in range(1,len(w1[i])):
        w1_save += f",{w1[i][x]}"
    w1_save +='\n'
with open('w1.txt','w') as file:
    file.write(w1_save)

w2_save = ''
for i in range(len(w2)):
    w2_save += f"{w2[i][0]}" 
    for x in range(1,len(w2[i])):
        w2_save += f",{w2[i][x]}"
    w2_save +='\n'
with open('w2.txt','w') as file:
    file.write(w2_save)
    
b1_save = ''
b1_save += f"{b1[0]}" 
for x in range(1,len(b1)):
    b1_save += f",{b1[x]}"
with open('b1.txt','w') as file:
    file.write(b1_save)

b2_save = ''
b2_save += f"{b2[0]}" 
for x in range(1,len(b2)):
    b2_save += f",{b2[x]}"
with open('b2.txt','w') as file:
    file.write(b2_save)

In [91]:
www1 = []
with open('w1.txt','r') as file:
    for line in file:
        # print(line)
        lista = line.strip().split(',')
        wwww1 = []
        for value in lista:
            wwww1.append(float(value))
        www1.append(wwww1)
print(www1 == w1)
www2 = []
with open('w2.txt','r') as file:
    for line in file:
        # print(line)
        lista = line.strip().split(',')
        wwww2 = []
        for value in lista:
            wwww2.append(float(value))
        www2.append(wwww2)
print(www2 == w2)


bb1 = []
with open('b1.txt','r') as file:
    for line in file:
        # print(line)
        lista = line.strip().split(',')
        for value in lista:
            bb1.append(float(value))
print(bb1 == b1)

bb2 = []
with open('b2.txt','r') as file:
    for line in file:
        # print(line)
        lista = line.strip().split(',')
        for value in lista:
            bb2.append(float(value))
print(bb2 == b2)




True
True
True
True


In [56]:
w1

[[-0.020000000000000018,
  0.09999999999999998,
  -0.10999999999999999,
  -0.09999999999999998,
  0.31000000000000005,
  -0.37,
  0.38,
  0.31000000000000005,
  -0.38,
  -0.15000000000000002,
  0.31999999999999995,
  -0.38,
  0.3300490448555415,
  -0.059950955144459424,
  -0.19,
  0.030000000000000027,
  -0.42,
  0.33999999999999997,
  0.14,
  -0.35,
  0.26,
  0.38,
  0.35,
  -0.41000000000000003,
  0.30000000000000004,
  -0.21000000000000002,
  -0.10999999999999999,
  -0.15999999999999998,
  0.06999999999999995,
  0.22999999999999998,
  -0.25,
  0.16000000000000003,
  -0.15999999999999998,
  0.35000001797599606,
  0.00011732904894545183,
  -0.09952364104290368,
  -0.3392823564233416,
  -0.15907913975136959,
  0.3107773461155247,
  -0.14944523804055512,
  0.1806700247102709,
  -0.3095565033536187,
  -0.0997894831642588,
  0.06002761279001314,
  0.05002460484859628,
  0.34002587347263213,
  0.3100258734726322,
  -0.38,
  -0.39994399713313067,
  0.36005600286686934,
  0.4200792936113079,

In [57]:
w2

[[-0.018765348106283178,
  -0.012114532888972003,
  0.11808527412312961,
  -0.10133498991031463,
  0.09738881307943695,
  -0.12916669082510118,
  0.1678901226309622,
  -0.3606163258003461,
  -0.3234120342257051,
  0.5146670304759029,
  0.2798225136285832,
  0.33394890085601586,
  0.04788076502448862,
  0.157877217215495,
  -0.295292052562098,
  -0.6933803560273053,
  -0.25484157395193224,
  -0.15203499720232794,
  -0.21730216570881658,
  -0.047280789487037234,
  0.31601999003104325,
  0.2880497089839436,
  0.2228107991522236,
  0.26008766461811456,
  0.08901874035736058,
  0.034973695723994724,
  0.04967021384552691,
  -0.11145285648368618,
  0.2918959935053464,
  -0.15779136035849026,
  0.03997913867306672,
  -0.23843110391675237,
  -0.23017143443959107,
  -0.14607987795513544,
  0.23464931051693275,
  -0.06319404720064231,
  -0.10864179773061651,
  0.0021438513756244486,
  0.32771097524082377,
  -0.28378334847828907],
 [0.20277513167383343,
  -0.3516039768427905,
  0.0344811367754040

In [58]:
b1

[0.13363421870586087,
 -0.16636578129423538,
 -0.45636578129434874,
 0.4336342187056981,
 0.453634218705706,
 0.013634218705829548,
 -0.28636578129426854,
 -0.23636578129420524,
 -0.32636578129426935,
 -0.15636578129422293,
 0.4336342187056981,
 0.5236342187057409,
 0.013634218705829548,
 0.3436342187057984,
 -0.27636578129428485,
 0.18363421870585536,
 0.4736342187057098,
 0.3136342187058226,
 -0.006365781294167498,
 -0.3363657812942849,
 0.2236342187058439,
 0.38363421870575004,
 -0.17636578129424121,
 -0.15636578129422293,
 0.30363421870580404,
 0.19363421870584904,
 0.14363421870585721,
 -0.17636578129424121,
 -0.4263657812942963,
 0.3936342187057225,
 -0.4463657812943422,
 0.08363421870583763,
 -0.38636578129430815,
 0.11363421870585642,
 0.54363421870579,
 -0.32636578129426935,
 0.4336342187056981,
 0.32363421870581616,
 -0.22636578129420373,
 0.4336342187056981]

In [59]:
b2

[0.43999999999999995,
 0.22999999999999998,
 -0.33999999999999997,
 -0.020000000000000018,
 0.06999999999999995,
 0.42000000000000004,
 -0.07,
 -0.09999999999999998,
 -0.18,
 0.16000000000000003]

In [64]:
ww2 = [[-0.018765348106283178,
  -0.012114532888972003,
  0.11808527412312961,
  -0.10133498991031463,
  0.09738881307943695,
  -0.12916669082510118,
  0.1678901226309622,
  -0.3606163258003461,
  -0.3234120342257051,
  0.5146670304759029,
  0.2798225136285832,
  0.33394890085601586,
  0.04788076502448862,
  0.157877217215495,
  -0.295292052562098,
  -0.6933803560273053,
  -0.25484157395193224,
  -0.15203499720232794,
  -0.21730216570881658,
  -0.047280789487037234,
  0.31601999003104325,
  0.2880497089839436,
  0.2228107991522236,
  0.26008766461811456,
  0.08901874035736058,
  0.034973695723994724,
  0.04967021384552691,
  -0.11145285648368618,
  0.2918959935053464,
  -0.15779136035849026,
  0.03997913867306672,
  -0.23843110391675237,
  -0.23017143443959107,
  -0.14607987795513544,
  0.23464931051693275,
  -0.06319404720064231,
  -0.10864179773061651,
  0.0021438513756244486,
  0.32771097524082377,
  -0.28378334847828907],
 [0.20277513167383343,
  -0.3516039768427905,
  0.03448113677540405,
  -0.0033204997810051197,
  0.2085257070196301,
  -0.02939207958278245,
  -0.3351915183873804,
  0.37405510893634475,
  -0.2208047155524982,
  -0.14409572949885346,
  -0.4618134419738795,
  0.6358476093556574,
  0.313740715891006,
  0.06732745698953832,
  0.17910581861173358,
  0.3635647044777589,
  0.47229087687620086,
  0.38161107073089595,
  0.006578499129284972,
  0.41324760426089624,
  0.41149126161822747,
  -0.17569368973852262,
  0.03941430187599533,
  0.050763224636281676,
  -0.36815209021820133,
  -0.0519179277751921,
  -0.33797779023874125,
  -0.2259611850633172,
  0.20968435685350217,
  -0.06684279836148355,
  -0.0905699318396144,
  0.20858481794933648,
  -0.19993915446649402,
  -0.004909471893045392,
  -0.4043740076237942,
  -0.40321053350444547,
  -0.019889984341349667,
  -0.18641637648086182,
  -0.00590033780024331,
  -0.11442124542494156],
 [-0.3339011331039488,
  0.16674654776223294,
  0.16823036511264228,
  -0.3460608739369205,
  -0.20898681016280762,
  -0.2659338942338446,
  0.3067190200388244,
  -0.1603553721662033,
  -0.05897539824710337,
  0.3704636514703213,
  0.0037895950424047015,
  -0.19067119930754028,
  0.19198280209577057,
  -0.25973023869816747,
  0.09379565471145636,
  0.06563028662126733,
  -0.16824749557460109,
  -0.25242088081879505,
  -0.22233041434483877,
  -0.07606761076446972,
  0.23952727947780864,
  -0.20698149244578487,
  -0.23095156571337624,
  -0.02227011226625374,
  0.09764591987077278,
  0.03377924896157981,
  -0.3242369885258077,
  -0.24897703951841987,
  0.2033764740768906,
  -0.1639889203447746,
  0.017508804343274217,
  0.10206271000731042,
  0.31220114069451604,
  0.07065995122424414,
  0.15725869617431829,
  -0.3837878655431467,
  -0.07617943843032085,
  0.5186855790746077,
  0.24239672390396436,
  0.15242380890017118],
 [0.13922504885907594,
  0.20761192606199616,
  0.24605005482826348,
  0.015519246786081691,
  -0.17303163300028795,
  0.10899636218501699,
  -0.30511235804155346,
  0.05312717555028668,
  0.03768133411913024,
  0.06538734645698477,
  0.022671786337689015,
  -0.1832487363757718,
  -0.32640793595476847,
  0.274918754083806,
  0.211584301460448,
  -0.22684781459487777,
  0.21445922443096666,
  0.12974032099215874,
  -0.2960139013060007,
  0.20834166621290653,
  -0.39749572833440555,
  0.05961711044073244,
  0.2163758969802902,
  0.1370219452401705,
  -0.31568611805394736,
  -0.24106731483284932,
  -0.11174862223975719,
  0.4049782733596904,
  -0.1962303171856602,
  -0.059490147265089346,
  -0.13378698165325267,
  -0.19249205935468505,
  0.17108319745192366,
  -0.3578397566537959,
  -0.3972026325476629,
  -0.3415653636802587,
  0.32725016485719643,
  0.18392189757073313,
  0.08562849173487883,
  0.04257495519014325],
 [-0.14527083476755492,
  0.04149860308890354,
  -0.1310953441175992,
  0.07409388633384575,
  -0.18610752189356894,
  -0.2922757787078022,
  0.3993653675998116,
  0.0014326351697325128,
  -0.419361441282873,
  -0.25692239089650964,
  -0.13468615568487677,
  0.3925430702578724,
  0.025859865361442837,
  -0.0800334131439176,
  -0.014655283509166018,
  -0.12299415923606798,
  0.28970187838669204,
  0.13703652824303772,
  0.3879228693730791,
  0.032120762846342736,
  0.2705384553744287,
  -0.1400432196701618,
  -0.3195248324061402,
  -0.3215497611882098,
  0.37705589931870814,
  0.42327743935561385,
  0.3873251904709483,
  -0.07954904522958618,
  0.13661460549587195,
  0.13502546867484452,
  0.2830312893628896,
  -0.019465695588273987,
  0.2418756336101655,
  -0.09757420420222787,
  0.03953236488307681,
  -0.09937315317199619,
  0.18943260437199755,
  -0.019945322160010086,
  -0.17354863419956462,
  0.3363033491408409],
 [-0.07305656825146448,
  0.1556776517221219,
  0.4626770275337466,
  0.18499099437159272,
  -0.43657208106517204,
  -0.15651265627886934,
  -0.004429285613793925,
  0.33895327345755283,
  -0.43535487197915396,
  0.009692332344654924,
  -0.21898110396604187,
  0.2503517541134421,
  0.2868159152142322,
  -0.37162753209582544,
  -0.26230797952348567,
  -0.3911091617271487,
  -0.3846900337828402,
  0.1338015166695075,
  0.48812754538136327,
  -0.059327652431079944,
  -0.3684841167804082,
  -0.32261723200629633,
  0.14062616206637302,
  0.27501811617058697,
  0.033092093526303566,
  -0.037269009086140045,
  0.26306186816773863,
  0.24771641343838696,
  -0.3898277726555579,
  0.2479403190016801,
  -0.5177392767338824,
  -0.12039114043777464,
  -0.2411399998558143,
  -0.18564092307466867,
  0.030428887668207595,
  0.13746679467911788,
  0.25368361714548615,
  0.3921826068994577,
  -0.3070159952865518,
  -0.33400929956850267],
 [-0.1301976215383517,
  -0.15893318710216014,
  0.04792509892381444,
  -0.17509605082940494,
  -0.49752016008671524,
  -0.5299509783653454,
  -0.4611588381343599,
  -0.23763308598909363,
  0.0939921591742828,
  0.3534955029741621,
  -0.35637887766344356,
  0.02712444892019555,
  0.3053823788478956,
  0.09743808255950163,
  -0.4174343754202484,
  -0.284101716529721,
  0.127501449797613,
  0.056380036075999666,
  -0.4795806184890661,
  -0.14642376786696096,
  0.20717579602122438,
  -0.03446429989113386,
  0.0971444460155692,
  -0.306952652303949,
  0.4807847428793783,
  -0.11099194040692428,
  0.1469872999034381,
  -0.2456473545580882,
  -0.47413267044391194,
  0.013967453398231367,
  -0.17038133497006888,
  0.25830809040602604,
  0.14982193863796348,
  0.18261893479650154,
  0.025349822596003872,
  -0.27023137128250485,
  0.11268538768635035,
  -0.2545155551354999,
  -0.06400228562862634,
  -0.42770719505588667],
 [0.020309694284416726,
  0.040710766318003924,
  -0.42415901105437,
  0.16476866947932342,
  0.16772994614636824,
  0.0209274870367678,
  -0.05299097761630046,
  0.11875319217724487,
  -0.06925614502322822,
  -0.014781325296132894,
  0.031132290685791902,
  0.008580742363506496,
  0.018179786381820034,
  0.17090229815850946,
  -0.10193903675769883,
  0.09446565761055092,
  -0.2926056053539584,
  0.2431101544024394,
  0.4508808083262638,
  0.3288774191091003,
  -0.05814511322213811,
  -0.34382954222615536,
  0.23861695994523885,
  0.5861278275041052,
  0.13353149820788393,
  0.11271572271628665,
  0.1374094596637091,
  -0.2859674887433439,
  -0.24119950543643665,
  -0.5782326147420854,
  0.32241352692187664,
  -0.458927880628568,
  0.24478702036859265,
  -0.5578306768113077,
  0.4452290013841154,
  0.2000101707112106,
  0.1114857040994742,
  0.03870525985832821,
  0.09028841886685246,
  0.08049187189553755],
 [0.06979387629048076,
  -0.17732126595533854,
  0.35111105811613963,
  -0.06250463096195008,
  -0.13627909567606994,
  -0.04856983550917512,
  -0.43521322591569106,
  0.07655142020174886,
  -0.19871345139995295,
  0.20454129827147796,
  0.12508774894905694,
  -0.015728130073395883,
  -0.33658302322833644,
  -0.3570115883957827,
  -0.048913077143681864,
  0.10253786868340686,
  -0.2908458329660892,
  -0.29973158454328347,
  -0.21182177025365667,
  0.24059169838478345,
  0.16428108982270476,
  0.29074881268705655,
  -0.3167395027225543,
  -0.20593117266568134,
  0.22266558733054936,
  -0.1882705000062097,
  0.05349982167642492,
  -0.051192293546848726,
  0.15015364051027774,
  -0.17386553845877367,
  0.014091599045821203,
  -0.06106536407322152,
  -0.18452484317249168,
  -0.20269750147376212,
  0.17256051287201546,
  0.23118228438908828,
  0.15738029502194012,
  0.3722321756432215,
  -0.08573235628797303,
  -0.42092181375748744],
 [-0.13091224533998994,
  -0.022272532163744585,
  -0.2533056602410999,
  -0.061055751551302634,
  -0.10514716436076443,
  0.011878064281219465,
  -0.13987830656055728,
  0.06573197846299345,
  -0.03579543558284026,
  -0.33244771630193665,
  0.25935564464495997,
  0.12125153989008783,
  -0.46685126963356866,
  -0.060061036673144046,
  -0.12394396986720645,
  -0.017765309277744806,
  0.35727711213794566,
  0.3125078354504016,
  -0.14646085210759224,
  -0.2740793302643543,
  -0.18490891400845055,
  0.1552138438662877,
  -0.2477726651936581,
  0.14768492025485924,
  0.15004372678125677,
  0.12477058534996524,
  0.46600954727670646,
  -0.023947423654755724,
  -0.3603348047203301,
  -0.24672186154408646,
  0.14545316684987325,
  -0.018182374363392078,
  0.1560065011713014,
  -0.3907064739567233,
  -0.03343195592311634,
  0.45270308460372244,
  -0.10720655268012931,
  0.20300588335447176,
  0.22017499945626143,
  0.23904891715837193]]
ww2 == w2

True

In [63]:
a2 = [0.43999999999999995,
 0.22999999999999998,
 -0.33999999999999997,
 -0.020000000000000018,
 0.06999999999999995,
 0.42000000000000004,
 -0.07,
 -0.09999999999999998,
 -0.18,
 0.16000000000000003]

In [60]:
a1 = [0.13363421870586087,
 -0.16636578129423538,
 -0.45636578129434874,
 0.4336342187056981,
 0.453634218705706,
 0.013634218705829548,
 -0.28636578129426854,
 -0.23636578129420524,
 -0.32636578129426935,
 -0.15636578129422293,
 0.4336342187056981,
 0.5236342187057409,
 0.013634218705829548,
 0.3436342187057984,
 -0.27636578129428485,
 0.18363421870585536,
 0.4736342187057098,
 0.3136342187058226,
 -0.006365781294167498,
 -0.3363657812942849,
 0.2236342187058439,
 0.38363421870575004,
 -0.17636578129424121,
 -0.15636578129422293,
 0.30363421870580404,
 0.19363421870584904,
 0.14363421870585721,
 -0.17636578129424121,
 -0.4263657812942963,
 0.3936342187057225,
 -0.4463657812943422,
 0.08363421870583763,
 -0.38636578129430815,
 0.11363421870585642,
 0.54363421870579,
 -0.32636578129426935,
 0.4336342187056981,
 0.32363421870581616,
 -0.22636578129420373,
 0.4336342187056981]

In [62]:
a1 == b1

True

In [65]:
a2 ==  b2

True

In [ ]:
X_test = DAT[42000:]
y_test = RES[42000:]
number = 0
for i in range(len(y_test)):  
  X = X_test[i]
  E = y_test[i]
  _, _,_, A2 = forward(X,www1, bb1, www2, bb2)
  if A2.index(max(A2)) == E.index(max(E)):
    number +=1
print(number/len(y_test))
